# Using Amazon Elastic Inference with TensorFlow on an Amazon SageMaker Notebook Instance

This notebook demonstrates how to enable and utilize Amazon Elastic Inference with our predefined SageMaker TensorFlow containers on your SageMaker notebook instance. This notebook will train locally on your notebook instance and make inferences to the EI accelerator attached to your notebook instance.

Amazon Elastic Inference (EI) is a resource you can attach to your Amazon EC2 instances to accelerate your deep learning (DL) inference workloads. EI allows you to add inference acceleration to an Amazon SageMaker hosted endpoint or Jupyter notebook for a fraction of the cost of using a full GPU instance. Since EI is only meant for inferences, no training logic changes are needed. For more information please visit: https://docs.aws.amazon.com/sagemaker/latest/dg/ei.html

This notebook is an adaption of the [SageMaker TensorFlow Iris DNN classifier](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_iris_dnn_classifier_using_estimators/tensorflow_iris_dnn_classifier_using_estimators.ipynb), with changes showing the easy changes needed to enable and use EI with TensorFlow on SageMaker.

1. [The Iris dataset](#The-Iris-dataset)
1. [Setup](#Setup)
1. [tf.estimator](#tf.estimator)
1. [Construct a deep neural network classifier](#Construct-a-deep-neural-network-classifier)
    1. [Complete neural network source code](#Complete-neural-network-source-code)
    1. [Using a tf.estimator in SageMaker](#Using-a-tf.estimator-in-SageMaker)
    1. [Describe the training input pipeline](#Describe-the-training-input-pipeline)
    1. [Describe the serving input pipeline](#Describe-the-serving-input-pipeline)
1. [Train a Model on Amazon SageMaker using TensorFlow custom code](#Train-a-Model-on-Amazon-SageMaker-using-TensorFlow-custom-code)
1. [Deploy the trained Model to an Endpoint with an attached EI accelerator](#Deploy-the-trained-Model-to-an-Endpoint-with-an-attached-EI-accelerator)
    1. [Using EI with a SageMaker notebook instance](#Using-EI-with-a-SageMaker-notebook-instance)
    1. [Invoke the Endpoint to get inferences locally](#Invoke-the-Endpoint-to-get-inferences-locally)
    1. [Delete the Endpoint](#Delete-the-Endpoint)

If you are familiar with SageMaker and already have a trained model, skip ahead to the [Creating-an-inference-endpoint section](#Creating-an-inference-endpoint-with-EI)

For this example, we will be utilizing the SageMaker Python SDK, which helps deploy your models for training and hosting in SageMaker.

In this tutorial, you'll use tf.estimator to construct a
[neural network](https://en.wikipedia.org/wiki/Artificial_neural_network)
classifier and train it on the
[Iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set) to
predict flower species based on sepal/petal geometry. You'll write code to
perform the following five steps:

1.  Deploy a TensorFlow container in SageMaker
2.  Load CSVs containing Iris training/test data from a S3 bucket into a TensorFlow `Dataset`
3.  Construct a `tf.estimator.DNNClassifier` neural network classifier
4.  Train the model using the training data
5.  Host the model in an endpoint with EI
6.  Classify new samples invoking the endpoint

This tutorial is a simplified version of TensorFlow's [get_started/estimator](https://www.tensorflow.org/get_started/estimator#fit_the_dnnclassifier_to_the_iris_training_data) tutorial but using SageMaker and the SageMaker Python SDK to simplify training and hosting.

## The Iris dataset

The [Iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set) contains
150 rows of data, comprising 50 samples from each of three related Iris species:
*Iris setosa*, *Iris virginica*, and *Iris versicolor*.

![Petal geometry compared for three iris species: Iris setosa, Iris virginica, and Iris versicolor](https://www.tensorflow.org/images/iris_three_species.jpg) **From left to right,
[*Iris setosa*](https://commons.wikimedia.org/w/index.php?curid=170298) (by
[Radomil](https://commons.wikimedia.org/wiki/User:Radomil), CC BY-SA 3.0),
[*Iris versicolor*](https://commons.wikimedia.org/w/index.php?curid=248095) (by
[Dlanglois](https://commons.wikimedia.org/wiki/User:Dlanglois), CC BY-SA 3.0),
and [*Iris virginica*](https://www.flickr.com/photos/33397993@N05/3352169862)
(by [Frank Mayfield](https://www.flickr.com/photos/33397993@N05), CC BY-SA
2.0).**

Each row contains the following data for each flower sample:
[sepal](https://en.wikipedia.org/wiki/Sepal) length, sepal width,
[petal](https://en.wikipedia.org/wiki/Petal) length, petal width, and flower
species. Flower species are represented as integers, with 0 denoting *Iris
setosa*, 1 denoting *Iris versicolor*, and 2 denoting *Iris virginica*.

Sepal Length | Sepal Width | Petal Length | Petal Width | Species
:----------- | :---------- | :----------- | :---------- | :-------
5.1          | 3.5         | 1.4          | 0.2         | 0
4.9          | 3.0         | 1.4          | 0.2         | 0
4.7          | 3.2         | 1.3          | 0.2         | 0
&hellip;     | &hellip;    | &hellip;     | &hellip;    | &hellip;
7.0          | 3.2         | 4.7          | 1.4         | 1
6.4          | 3.2         | 4.5          | 1.5         | 1
6.9          | 3.1         | 4.9          | 1.5         | 1
&hellip;     | &hellip;    | &hellip;     | &hellip;    | &hellip;
6.5          | 3.0         | 5.2          | 2.0         | 2
6.2          | 3.4         | 5.4          | 2.3         | 2
5.9          | 3.0         | 5.1          | 1.8         | 2

For this tutorial, the Iris data has been randomized and split into two separate
CSVs:

*   A training set of 120 samples
    iris_training.csv
*   A test set of 30 samples
    iris_test.csv

These files are provided in the SageMaker sample data bucket:
**s3://sagemaker-sample-data-{region}/tensorflow/iris**. Copies of the bucket exist in each SageMaker region. When we access the data, we'll replace {region} with the AWS region the notebook is running in.

## Setup

Let's start by creating a SageMaker session and specifying the IAM role arn used to give training and hosting access to your data. See the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-tensorflow-iris'

role = sagemaker.get_execution_role()

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments. Just change your estimator's train_instance_type to local or local_gpu. For more information, see [local mode](https://github.com/aws/sagemaker-python-sdk#local-mode).

To use Amazon Elastic Inference locally change your `accelerator_type` to `local_sagemaker_notebook` when calling `deploy()`.

***`local_sagemaker_notebook` will only work if you created your notebook instance with an EI accelerator attached to it.***

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU). Running following script will install docker-compose or nvidia-docker-compose and configure the notebook environment for you.

Note, you can only run a single local notebook at a time.

In [ ]:
!/bin/bash ./setup.sh

# tf.estimator

The tf.estimator framework makes it easy to construct and train machine learning models via its high-level Estimator API. Estimator offers classes you can instantiate to quickly configure common model types such as regressors and classifiers:


*   **```tf.estimator.LinearClassifier```**:
    Constructs a linear classification model.
*   **```tf.estimator.LinearRegressor```**:
    Constructs a linear regression model.
*   **```tf.estimator.DNNClassifier```**:
    Construct a neural network classification model.
*   **```tf.estimator.DNNRegressor```**:
    Construct a neural network regression model.
*   **```tf.estimator.DNNLinearCombinedClassifier```**:
    Construct a neural network and linear combined classification model.
*   **```tf.estimator.DNNRegressor```**:
    Construct a neural network and linear combined regression model.
    
More information about estimators can be found [here](https://www.tensorflow.org/extend/estimators)

# Construct a deep neural network classifier

## Complete neural network source code 

Here is the full code for the neural network classifier:

In [ ]:
!cat "iris_dnn_classifier.py"

With few lines of code, using SageMaker and TensorFlow, you can create a deep neural network model, ready for training and hosting. Let's give a deeper look at the code.

### Using a tf.estimator in SageMaker
Using a TensorFlow estimator in SageMaker is very easy, you can create one with few lines of code:

In [ ]:
def estimator(model_path, hyperparameters):
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[4])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 10],
                                      n_classes=3,
                                      model_dir=model_path)

The code above first defines the model's feature columns, which specify the data
type for the features in the data set. All the feature data is continuous, so
`tf.feature_column.numeric_column` is the appropriate function to use to
construct the feature columns. There are four features in the data set (sepal
width, sepal height, petal width, and petal height), so accordingly `shape`
must be set to `[4]` to hold all the data.

Then, the code creates a `DNNClassifier` model using the following arguments:

*   `feature_columns=feature_columns`. The set of feature columns defined above.
*   `hidden_units=[10, 20, 10]`. Three
    [hidden layers](http://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw),
    containing 10, 20, and 10 neurons, respectively.
*   `n_classes=3`. Three target classes, representing the three Iris species.
*   `model_dir=model_path`. The directory in which TensorFlow will save
    checkpoint data during model training. 

### Describe the training input pipeline

The `tf.estimator` API uses input functions, which create the TensorFlow
operations that generate data for the model.
We can use `tf.estimator.inputs.numpy_input_fn` to produce the input pipeline:

In [ ]:
def train_input_fn(training_dir, hyperparameters):
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=os.path.join(training_dir, 'iris_training.csv'),
        target_dtype=np.int,
        features_dtype=np.float32)

    return tf.estimator.inputs.numpy_input_fn(
        x={INPUT_TENSOR_NAME: np.array(training_set.data)},
        y=np.array(training_set.target),
        num_epochs=None,
        shuffle=True)()

### Describe the serving input pipeline

After traininng your model, SageMaker will host it in a TensorFlow serving. You need to describe a serving input function:

In [ ]:
def serving_input_fn(hyperparameters):
    feature_spec = {INPUT_TENSOR_NAME: tf.FixedLenFeature(dtype=tf.float32, shape=[4])}
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)()

Now we are ready to submit the script for training.

# Train a Model on Amazon SageMaker using TensorFlow custom code

We can use the SDK to run our training script locally.

1. Pass the path to the `iris_dnn_classifier.py` file, which contains the functions for defining your estimator, to the sagemaker.TensorFlow init method.
2. Pass the S3 location that we uploaded our data to previously to the `fit()` method.
3. Pass `local` into the `train_instance_type`. By passing local, training will be done inside of a Docker container on this notebook instance.

In [ ]:
from sagemaker.tensorflow import TensorFlow

iris_estimator = TensorFlow(entry_point='iris_dnn_classifier.py',
                            role=role,
                            framework_version='1.12',
                            train_instance_count=1,
                            train_instance_type='local',
                            training_steps=1000,
                            evaluation_steps=100)

In [ ]:
%%time
import boto3

# use the region-specific sample data bucket
region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/tensorflow/iris'.format(region)

iris_estimator.fit(train_data_location)

# Deploy the trained Model to an Endpoint with an attached EI accelerator

The `deploy()` method creates an endpoint which serves prediction requests in real-time.

The only change required for utilizing EI with our SageMaker TensorFlow containers only requires providing an `accelerator_type` parameter, which determines which type of EI accelerator to attach to your endpoint. The supported types of accelerators can be found here: https://aws.amazon.com/sagemaker/pricing/instance-types/

No code changes are necessary for your model, as our predefined TensorFlow containers utilizes TensorFlow serving, which has been modified to utilize EI for inference, as long as an EI accelerator is attached to the endpoint.

## Using EI with a SageMaker notebook instance

Here we're going to utilize the EI accelerator attached to our local SageMaker notebook instance. This can be done by using `local_sagemaker_notebook` as the value for `accelerator_type`. This will make an inference request against the TensorFlow Serving endpoint running on this Notebook Instance with an attached EI.

An EI accelerator must be attached in order to make inferences using EI.

As of now, an EI accelerator attached to a notebook will initialize for the first deep learning framework used to inference against EI. If you wish to use EI with another deep learning framework, please either restart or create a new notebook instance with the new EI.

***`local_sagemaker_notebook` will only work if you created your notebook instance with an EI accelerator attached to it.*** 

***Please restart or create a new notebook instance if you wish to use EI with a different framework than the first framework used on this notebook instance as specified when calling `deploy()` with `local_sagemaker_notebook`for `accelerator_type`.***

In [ ]:
%%time
iris_predictor = iris_estimator.deploy(initial_instance_count=1,
                                       instance_type='local',
                                       accelerator_type='local_sagemaker_notebook')

#  Invoke the Endpoint to get inferences locally

Invoking prediction:

In [ ]:
%%time
iris_predictor.predict([6.4, 3.2, 4.5, 1.5]) #expected label to be 1

# Delete the Endpoint

After you have finished with this example, remember to delete the prediction endpoint.

In [ ]:
print(iris_predictor.endpoint)

In [ ]:
import sagemaker

iris_predictor.delete_endpoint()